In [39]:
from gurobipy import GRB
import gurobipy as gb
import pandas as pd
import numpy as np

In [40]:
# Create a new model
model = gb.Model('VaccineTransportation')

In [41]:
# Create the a single class of decision variables where
# From = {Billy} and To = {29 sites}.
x = model.addVars(29, lb=0, vtype=GRB.CONTINUOUS, name="Billy Shipping")
# From = {Pearson} and To = {29 sites}.
y = model.addVars(29, lb=0, vtype=GRB.CONTINUOUS, name="Pearson Shipping")

In [42]:
# Objective function
Billy_Bishop_Toronto_City_Airport_costs = [0.05, 0.05, 0.05, 0.05, 0.05, 0.06, 0.06, 0.06, 0.06, 0.06, 0.07, 0.07, 0.07, 0.07, 0.07, 0.08, 0.08, 0.08, 0.08, 0.08, 0.09, 0.09, 0.09, 0.09, 0.09, 0.1, 0.1, 0.1, 0.1]
Toronto_Pearson_Airport_costs = [0.08, 0.08, 0.08, 0.08, 0.08, 0.05, 0.05, 0.05, 0.05, 0.05, 0.09, 0.09, 0.09, 0.09, 0.09, 0.1, 0.1, 0.1, 0.1, 0.1, 0.07, 0.07, 0.07, 0.07, 0.07, 0.06, 0.06, 0.06, 0.06]

In [43]:
#Set Objectives
Billy_objective = gb.quicksum(Billy_Bishop_Toronto_City_Airport_costs[j] * x[j] for j in range(29))
Pearson_objective = gb.quicksum(Toronto_Pearson_Airport_costs[j] * y[j] for j in range(29))
model.setObjective(Billy_objective + Pearson_objective, GRB.MINIMIZE)

In [44]:
# Constraints

# Difference constraint for sites 1-5
model.addConstr(gb.quicksum(x[j] for j in range(5)) - gb.quicksum(y[j] for j in range(5)) <= 4800, name="4800 constraint")
model.addConstr(gb.quicksum(y[j] for j in range(5)) - gb.quicksum(x[j] for j in range(5)) <= 4800, name="4800 constraint negative")

# Pearson to sites 21-25 constraint
model.addConstr(gb.quicksum(y[j] for j in range(20,25)) <= 8 * (gb.quicksum(x[j] for j in range(10,15))), name="Second constraint")

# Billy Bishop to sites 26-29 constraint
model.addConstr(gb.quicksum(x[j] for j in range(25,29)) >= 0.8 * (gb.quicksum(y[j] for j in range(15,20))), name="Third constraint")

# Supply constraints
model.addConstr(gb.quicksum(x[j] for j in range(29)) == 100000, "Billy Supply Constraint")
model.addConstr(gb.quicksum(y[j] for j in range(29)) == 250000, "Pearson Supply Constraint")

#Hosipital Constraint
# 50000 = 7x + 22x/4
# x = 4000
# 4000 per day, 7 days = 28000 per week
for j in range(7):
    model.addConstr(x[j] + y[j] <= 28000, f"Hospital Constraint_{j}")

# 50000 = 4*7x + 22x
# x = 1000
# 1000 per day, 7 days = 7000 per week
for j in range(7,29):
    model.addConstr(x[j] + y[j] <= 7000, f"Non-Hospital Constraint_{j}")

In [45]:
# Optimize the model
model.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (22631.2))

CPU model: 13th Gen Intel(R) Core(TM) i7-13700H, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 35 rows, 58 columns and 155 nonzeros
Model fingerprint: 0xc46d81a7
Coefficient statistics:
  Matrix range     [8e-01, 8e+00]
  Objective range  [5e-02, 1e-01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+03, 3e+05]
Presolve removed 1 rows and 0 columns
Presolve time: 0.00s
Presolved: 34 rows, 59 columns, 146 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.7500000e+04   8.539300e+04   0.000000e+00      0s
      56    2.4828000e+04   0.000000e+00   0.000000e+00      0s

Solved in 56 iterations and 0.01 seconds (0.00 work units)
Optimal objective  2.482800000e+04


In [51]:
# Print the decision variables
print(model.printAttr('X'))


    Variable            X 
-------------------------
Billy Shipping[0]        28000 
Billy Shipping[2]        28000 
Billy Shipping[3]        11600 
Billy Shipping[10]         4375 
Billy Shipping[19]          125 
Billy Shipping[25]         7000 
Billy Shipping[26]         7000 
Billy Shipping[27]         7000 
Billy Shipping[28]         6900 
Pearson Shipping[1]        28000 
Pearson Shipping[3]        16400 
Pearson Shipping[4]        28000 
Pearson Shipping[5]        28000 
Pearson Shipping[6]        28000 
Pearson Shipping[7]         7000 
Pearson Shipping[8]         7000 
Pearson Shipping[9]         7000 
Pearson Shipping[10]         2625 
Pearson Shipping[11]         7000 
Pearson Shipping[12]         7000 
Pearson Shipping[13]         7000 
Pearson Shipping[14]         7000 
Pearson Shipping[15]         7000 
Pearson Shipping[16]         7000 
Pearson Shipping[17]         7000 
Pearson Shipping[18]         7000 
Pearson Shipping[19]         6875 
Pearson Shipping[20]         7

In [52]:
print(model.ObjVal)

24828.0


In [47]:
#Print Shadow Prices
print(f"-"*50)
for constr in model.getConstrs():
        print(f"Constraint: {constr.ConstrName}, Shadow Price: {constr.Pi}")

--------------------------------------------------
Constraint: 4800 constraint, Shadow Price: 0.0
Constraint: 4800 constraint negative, Shadow Price: -0.11500000000000003
Constraint: Second constraint, Shadow Price: -0.030000000000000013
Constraint: Third constraint, Shadow Price: 0.3000000000000001
Constraint: Billy Supply Constraint, Shadow Price: 0.08
Constraint: Pearson Supply Constraint, Shadow Price: 0.3400000000000001
Constraint: Hospital Constraint_0, Shadow Price: -0.14500000000000002
Constraint: Hospital Constraint_1, Shadow Price: -0.14500000000000005
Constraint: Hospital Constraint_2, Shadow Price: -0.14500000000000002
Constraint: Hospital Constraint_3, Shadow Price: -0.14500000000000002
Constraint: Hospital Constraint_4, Shadow Price: -0.14500000000000005
Constraint: Hospital Constraint_5, Shadow Price: -0.2900000000000001
Constraint: Hospital Constraint_6, Shadow Price: -0.2900000000000001
Constraint: Non-Hospital Constraint_7, Shadow Price: -0.2900000000000001
Constraint